# Enunciado
Dos péndulos de longitud $l$ y masa $m$ están separados una distancia $2d$ y unidos por un resorte de longitud natural también $2d$ y constante $k$. Voy a encontrar las posiciones de equilibrio, y las ecuaciones de movimiento para pequeñas oscilaciones alrededor del punto de equilibrio estable.

### Definiendo las variables y armando el lagrangiano

In [1]:
import sympy as smp

In [2]:
d, l, k, m, g, t = smp.symbols('d l k m g t')
the1, the2 = smp.symbols(r'\theta_1 \theta_2', cls=smp.Function)
the1 = the1(t)
the2 = the2(t)
the1_t = smp.diff(the1,t)
the2_t = smp.diff(the2,t)
the1_tt = smp.diff(the1_t,t)
the2_tt = smp.diff(the2_t,t)

In [3]:
x1 = -d + l*smp.sin(the1)
x2 = d + l*smp.sin(the2)
y1 = - l*smp.cos(the1)
y2 = - l*smp.cos(the2)

T = smp.Rational(1,2)*m*(smp.diff(x1,t)**2+smp.diff(y1,t)**2+smp.diff(x2,t)**2+smp.diff(y2,t)**2)
U = m*g*(y1+y2) + smp.Rational(1,2)*k*(2*d-(x2-x1))**2
L = T-U

In [4]:
U.simplify()

l*(-2*g*m*(cos(\theta_1(t)) + cos(\theta_2(t))) + k*l*(sin(\theta_1(t)) - sin(\theta_2(t)))**2)/2

### Encontrando los puntos de equilibrio
Para eso vamos a resolver 
$$
\vec\nabla U = \vec0
$$

In [5]:
var = [the1, the2]
grad = [smp.diff(U,z).simplify() for z in var]
grad

[l*(g*m*sin(\theta_1(t)) + k*l*(sin(\theta_1(t)) - sin(\theta_2(t)))*cos(\theta_1(t))),
 l*(g*m*sin(\theta_2(t)) - k*l*(sin(\theta_1(t)) - sin(\theta_2(t)))*cos(\theta_2(t)))]

### Bueno
Por alguna razón `sympy` no puede resolver este sistema que se resuelve al menos con $\theta_1 = \theta_2 = 0 \lor \pi$. Así que voy a armar mi array de puntos críticos a manopla.

In [48]:
crit = smp.solve([smp.tan(the1)+smp.tan(the2), smp.tan(the2)-k*l/(m*g)*(smp.sin(the1)-smp.sin(the2))], var)
crit

[(0, 0),
 (0, pi),
 (I*atanh(tanh(log(-(g*m - sqrt(g**2*m**2 - 4*k**2*l**2))/(2*k*l)))),
  -I*log(-(g*m - sqrt(g**2*m**2 - 4*k**2*l**2))/(2*k*l))),
 (I*atanh(tanh(log(-(g*m + sqrt(g**2*m**2 - 4*k**2*l**2))/(2*k*l)))),
  -I*log(-(g*m + sqrt(g**2*m**2 - 4*k**2*l**2))/(2*k*l)))]

In [7]:
crit = [0, smp.pi]
crit

[0, pi]

In [18]:
hess = smp.MutableDenseMatrix([[smp.diff(grad[0], z) for z in var],[smp.diff(grad[1], z) for z in var]])

In [19]:
hess

Matrix([
[l*(g*m*cos(\theta_1(t)) - k*l*(sin(\theta_1(t)) - sin(\theta_2(t)))*sin(\theta_1(t)) + k*l*cos(\theta_1(t))**2),                                                                       -k*l**2*cos(\theta_1(t))*cos(\theta_2(t))],
[                                                                      -k*l**2*cos(\theta_1(t))*cos(\theta_2(t)), l*(g*m*cos(\theta_2(t)) + k*l*(sin(\theta_1(t)) - sin(\theta_2(t)))*sin(\theta_2(t)) + k*l*cos(\theta_2(t))**2)]])

In [15]:
hess.subs([(the1,crit[0]),(the2,crit[0])])

Matrix([
[l*(g*m + k*l),       -k*l**2],
[      -k*l**2, l*(g*m + k*l)]])

In [29]:
[hess[0].subs([(the1,crit[i]),(the2,crit[i])]) for i in [0,1]]

[l*(g*m + k*l), l*(-g*m + k*l)]

In [34]:
[hess.det().subs([(the1,crit[i]),(the2,crit[i])]) for i in [0,1]]

[-k**2*l**4 + l**2*(g*m + k*l)**2, -k**2*l**4 + l**2*(-g*m + k*l)**2]

Si bien podemos confirmar que para $\theta_1 = \theta_2 = 0$ tenemos un mínimo, porque tanto $H_{11}$ como su determinante son positivos, con el criterio de Sylvester no puedo decir que en $\theta_1 = \theta_2 = \pi$ tenga un máximo, porque depende de una relación entre $kl-gm$. Debería usar el criterio de los autovalores, pero eso queda para otro día. Hago la expansión alrededor de los $\theta = 0$

### Ecuaciones de movimiento

In [8]:
EL1 = smp.diff(smp.diff(L, the1_t), t) - smp.diff(L,the1)
EL2 = smp.diff(smp.diff(L, the2_t), t) - smp.diff(L,the2)

In [9]:
sols = smp.solve([EL1,EL2], [the1_tt,the2_tt])

In [10]:
sols[the1_tt]

-g*sin(\theta_1(t))/l - k*sin(2*\theta_1(t))/(2*m) + k*sin(\theta_2(t))*cos(\theta_1(t))/m

In [11]:
sols[the2_tt]

-g*sin(\theta_2(t))/l + k*sin(\theta_1(t))*cos(\theta_2(t))/m - k*sin(2*\theta_2(t))/(2*m)

Bueno, después de asumir que las oscilaciones son pequeñas, o sea que $\theta_i \approx 0$, y que $\theta_i = a_i e^{i\omega t}$, y reemplazando todo esto en las ecuaciones de movimiento que obtuve arriba gracias a EL llego al sistema de ecuaciones homogeneo $Wa = 0$.

In [35]:
w = smp.symbols(r'\omega')
W = smp.MutableDenseMatrix([[w**2-g/l-k/m, k/m],[k/m, w**2-g/l-k/m]])
W

Matrix([
[\omega**2 - g/l - k/m,                   k/m],
[                  k/m, \omega**2 - g/l - k/m]])

Para que el sistema tenga solución aparte de la trivial, el determinante de esta matriz tiene que ser no nulo

In [43]:
W.det()

-k**2/m**2 + (\omega**2 - g/l - k/m)**2

In [42]:
smp.solve(W.det(), w)

[-sqrt(g/l), sqrt(g/l), -sqrt(g/l + 2*k/m), sqrt(g/l + 2*k/m)]

Sustituyendo esto en el sistema de ecuaciones y resolviendo para $a$ me debería dar la información para imaginarme el movimiento que van a hacer. Es decir, los signos y las amplitudes relativas me dicen en qué sentido se van a mover y cuánto una respecto a la otra.

In [54]:
W1 = W.subs(w, smp.sqrt(g/l)).col_insert(2,smp.Matrix([0,0]))
W2 = W.subs(w, smp.sqrt(g/l+k/m)).col_insert(2,smp.Matrix([0,0]))

In [56]:
a1, a2 = smp.symbols(r'a_1 a_2')
smp.solve_linear_system(W1, a1, a2)

{a_1: a_2}

In [58]:
smp.solve_linear_system(W2, a1, a2)

{a_1: 0, a_2: 0}